In [2]:
import pandas as pd
import json

def extract_key_info(filepath):
    """
    Extracts key information from the specified IAC Assessment Excel file.

    Args:
        filepath (str): The path to the IAC Assessment Template Excel file.

    Returns:
        dict: A dictionary containing extracted data from the 'General Info',
              'Energy-Waste Info', and 'Recommendation Info' sheets.
              Returns None if the file cannot be read.
    """
    try:
        # --- 1. Extract from 'General Info' Sheet ---
        # This sheet has a key-value structure. We read the first two columns.
        general_info_df = pd.read_excel(filepath, sheet_name='General Info', header=None, usecols=[0, 1])
        
        # Clean up keys by removing colons and whitespace, then convert to a dictionary
        general_info_df[0] = general_info_df[0].str.replace(':', '').str.strip()
        general_info_dict = dict(zip(general_info_df[0], general_info_df[1]))

        # --- 2. Extract from 'Energy-Waste Info' Sheet ---
        # Skip the top header rows to get to the data table.
        energy_df = pd.read_excel(filepath, sheet_name='Energy-Waste Info', skiprows=2)

        # Select and rename relevant columns for clarity
        energy_df = energy_df[['Energy/Waste Info', 'Consumption', 'Cost']]
        energy_df.rename(columns={'Energy/Waste Info': 'Source'}, inplace=True)

        # Drop rows where the 'Source' is not specified (e.g., empty rows)
        energy_df.dropna(subset=['Source'], inplace=True)
        energy_info_list = energy_df.to_dict(orient='records')


        # --- 3. Extract from 'Recommendation Info' Sheet ---
        # Skip header rows to get to the recommendations table
        reco_df = pd.read_excel(filepath, sheet_name='Recommendation Info', skiprows=2)

        # Drop the first unnamed column which appears to be an index
        reco_df = reco_df.loc[:, ~reco_df.columns.str.contains('^Unnamed')]
        
        # Remove rows that are entirely empty, which often appear at the end of tables
        reco_df.dropna(how='all', inplace=True)
        recommendation_list = reco_df.to_dict(orient='records')

        # --- 4. Combine all extracted data into a single dictionary ---
        extracted_data = {
            "general_info": general_info_dict,
            "energy_and_waste_info": energy_info_list,
            "recommendations": recommendation_list
        }

        return extracted_data

    except FileNotFoundError:
        print(f"Error: The file at {filepath} was not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def save_to_json(data, output_filename="extracted_data.json"):
    """
    Saves the extracted data dictionary to a JSON file.

    Args:
        data (dict): The dictionary containing the extracted data.
        output_filename (str): The name of the output JSON file.
    """
    with open(output_filename, 'w') as f:
        json.dump(data, f, indent=4)
    print(f"Data successfully saved to {output_filename}")



In [ ]:

# --- Main execution block ---
if __name__ == "__main__":
    # IMPORTANT: Replace this with the actual path to your Excel file.
    # If the script and the Excel file are in the same folder, you can just use the filename.
    excel_file_path = 'IACAssessmentTemplate.v2.1.xlsx'
    
    assessment_data = extract_key_info(excel_file_path)
    
    if assessment_data:
        print("--- Successfully Extracted Data ---")
        
        # Print a summary of the extracted data
        print("\n--- General Info ---")
        for key, value in assessment_data['general_info'].items():
            print(f"{key}: {value}")
            
        print("\n--- Energy & Waste (First 3 entries) ---")
        for record in assessment_data['energy_and_waste_info'][:3]:
            print(record)

        print("\n--- Recommendations (First 2 entries) ---")
        for record in assessment_data['recommendations'][:2]:
            print(record)
        
        # Save the complete data to a structured JSON file
        save_to_json(assessment_data)

In [5]:


# If the script and the Excel file are in the same folder, you can just use the filename.
excel_file_path_1 = '/Users/afschowdhury/Code Local/itac-report-validator/docs/report1/IACAssessmentTemplate.v2.1.xlsx'

assessment_data = extract_key_info(excel_file_path_1)

if assessment_data:
    print("--- Successfully Extracted Data ---")
    
    # Print a summary of the extracted data
    print("\n--- General Info ---")
    for key, value in assessment_data['general_info'].items():
        print(f"{key}: {value}")
        
    print("\n--- Energy & Waste (First 3 entries) ---")
    for record in assessment_data['energy_and_waste_info'][:3]:
        print(record)

    print("\n--- Recommendations (First 2 entries) ---")
    for record in assessment_data['recommendations'][:2]:
        print(record)
    
    # Save the complete data to a structured JSON file
    save_to_json(assessment_data)

An error occurred: "None of [Index(['Energy/Waste Info', 'Consumption', 'Cost'], dtype='object')] are in the [columns]"


In [10]:
from excel_keyinfo_extractor import extract_excel_key_info
from pathlib import Path

In [11]:
key_info = extract_excel_key_info(excel_file_path_1, Path("tables_out"))

In [12]:
key_info

{'workbook': 'IACAssessmentTemplate.v2.1.xlsx',
 'sheets': {'General Info': {'key_values': [],
   'table_count': 1,
   'table_csv_files': ['tables_out/General_Info_table1.csv']},
  'Energy-Waste Info': {'key_values': [],
   'table_count': 1,
   'table_csv_files': ['tables_out/Energy-Waste_Info_table1.csv']},
  'Recommendation Info': {'key_values': [],
   'table_count': 1,
   'table_csv_files': ['tables_out/Recommendation_Info_table1.csv']},
  'Data': {'key_values': [],
   'table_count': 1,
   'table_csv_files': ['tables_out/Data_table1.csv']}}}

In [20]:
from pandas import DataFrame

data_table_path = Path("tables_out/Data_table1.csv")

df = pd.read_csv(data_table_path)


df_first_two_cols = df.iloc[:, :2]
# Drop rows where both columns are empty/null
df_first_two_cols_cleaned = df_first_two_cols.dropna(how='all')

# Also drop rows where both columns are empty strings
df_first_two_cols_cleaned = df_first_two_cols_cleaned[
    ~((df_first_two_cols_cleaned.iloc[:, 0].astype(str).str.strip() == '') & 
      (df_first_two_cols_cleaned.iloc[:, 1].astype(str).str.strip() == ''))
]

print(df_first_two_cols_cleaned.head(20))







                      GENERAL INFO Do not format this column
0             SIC Code: (4 Digits)                      3491
1           NAICS Code: (6 Digits)                    332911
2                 Annual Sales ($)                  35000000
3                   # of Employees                       120
4               Plant Area (sqft.)                    211185
5                Principle Product         Industrial Valves
6                Annual Production                      6200
7                 Production Units                         1
8           Production Hrs. Annual                      5616
9        Motor Horsepower Capacity                       952
10        Largest Motor Horsepower                        93
11          Steam Capacity(LBM/Hr)                         0
12        Max Steam Pressure(PSIG)                         0
13               Air Compressor HP                        50
14  Max Compressed Air Press(PSIG)                       120
15               Energy/

In [23]:
general_info_df = pd.read_csv('tables_out/General_Info_table1.csv')
# only first 2 columns
general_info_df = general_info_df.iloc[:, :2]
general_info_df






,GENERAL INFO,Do not format this column
0,SIC Code: (4 Digits),3491
1,NAICS Code: (6 Digits),332911
2,Annual Sales ($),35000000
3,# of Employees,120
4,Plant Area (sqft.),211185
5,Principle Product,Industrial Valves
6,Annual Production,6200
7,Production Units,1
8,Production Hrs. Annual,5616
9,Motor Horsepower Capacity,952


In [26]:
# Convert general_info_df to dictionary
general_info_dict = general_info_df.set_index(general_info_df.columns[0])[general_info_df.columns[1]].to_dict()
general_info_dict


{'SIC Code: (4 Digits)': '3491',
 'NAICS Code: (6 Digits)': '332911',
 'Annual Sales ($)': '35000000',
 '# of Employees': '120',
 'Plant Area (sqft.)': '211185',
 'Principle Product': 'Industrial Valves',
 'Annual Production': '6200',
 'Production Units': '1',
 'Production Hrs. Annual': '5616',
 'Motor Horsepower Capacity': '952',
 'Largest Motor Horsepower': '93',
 'Steam Capacity(LBM/Hr)': nan,
 'Max Steam Pressure(PSIG)': nan,
 'Air Compressor HP': '50',
 'Max Compressed Air Press(PSIG)': '120'}

In [25]:
energy_waste_df = pd.read_csv('tables_out/Energy-Waste_info_table1.csv')
energy_waste_df






,Energy/Waste Info,Consumption,Cost,Consumption_1,col_5,Cost_1,Unit Cost,col_8,col_9,col_10,col_11,col_12
0,Electrical Consumption,2763509.0,308828.0,2763509.0,kWh,308828,0.111752,per kWh,NaN,NaN,NaN,NaN
1,Electrical Demand,2483.0,8408.0,2483.0,kW months/yr,8408,3.386226,per kW months/yr,NaN,NaN,NaN,NaN
2,Other Electrical Fees,NaN,NaN,NaN,NaN,0,NaN,per,NaN,NaN,NaN,317236.0
3,Natural Gas,2325.0,12911.0,2325.0,MMBtu,12911,5.553118,per MMBtu,NaN,NaN,NaN,NaN
4,L.P.G.,208.0,10467.0,208.0,MMBtu,10467,50.322115,per MMBtu,NaN,NaN,NaN,NaN
5,#1 Fuel Oil,NaN,NaN,0.0,MMBtu,0,NaN,per MMBtu,NaN,NaN,NaN,NaN
6,#2 Fuel Oil,NaN,NaN,0.0,MMBtu,0,NaN,per MMBtu,NaN,NaN,NaN,340614.0
7,#4 Fuel Oil,NaN,NaN,0.0,MMBtu,0,NaN,per MMBtu,NaN,NaN,NaN,NaN
8,#6 Fuel Oil,NaN,NaN,0.0,MMBtu,0,NaN,per MMBtu,NaN,NaN,NaN,NaN
9,Coal,NaN,NaN,0.0,MMBtu,0,NaN,per MMBtu,NaN,NaN,NaN,NaN


In [29]:
# Create dictionary for energy_waste_df with main key and its parameters
energy_waste_dict = {}

for index, row in energy_waste_df.iterrows():
    main_key = row.iloc[0]  # First column as main key
    
    # Only process if main key is present and not NaN
    if pd.notna(main_key) and str(main_key).strip() != '':
        main_key = str(main_key).strip()
        
        # Collect all parameters (remaining columns) for this key
        parameters = {}
        for col_idx in range(1, len(row)):
            col_name = energy_waste_df.columns[col_idx]
            value = row.iloc[col_idx]
            
            # Only add parameter if value is present, not NaN, and not 0 for consumption-related columns
            if pd.notna(value) and str(value).strip() != '':
                # Skip if it's a consumption column and value is 0
                if 'consumption' in col_name.lower() and value == 0:
                    continue
                parameters[col_name] = value
        
        # Only add main key if it has at least one parameter
        if parameters:
            energy_waste_dict[main_key] = parameters

energy_waste_dict




{'Electrical Consumption': {'Consumption': 2763509.0,
  'Cost': 308828.0,
  'Consumption_1': 2763509.0,
  'col_5': 'kWh',
  'Cost_1': 308828,
  'Unit Cost': 0.1117521238396545,
  'col_8': 'per kWh'},
 'Electrical Demand': {'Consumption': 2483.0,
  'Cost': 8408.0,
  'Consumption_1': 2483.0,
  'col_5': 'kW months/yr',
  'Cost_1': 8408,
  'Unit Cost': 3.3862263391059204,
  'col_8': 'per kW months/yr'},
 'Other Electrical Fees': {'Cost_1': 0, 'col_8': 'per ', 'col_12': 317236.0},
 'Natural Gas': {'Consumption': 2325.0,
  'Cost': 12911.0,
  'Consumption_1': 2325.0,
  'col_5': 'MMBtu',
  'Cost_1': 12911,
  'Unit Cost': 5.553118279569892,
  'col_8': 'per MMBtu'},
 'L.P.G.': {'Consumption': 208.0,
  'Cost': 10467.0,
  'Consumption_1': 208.0,
  'col_5': 'MMBtu',
  'Cost_1': 10467,
  'Unit Cost': 50.32211538461539,
  'col_8': 'per MMBtu'},
 '#1 Fuel Oil': {'col_5': 'MMBtu', 'Cost_1': 0, 'col_8': 'per MMBtu'},
 '#2 Fuel Oil': {'col_5': 'MMBtu',
  'Cost_1': 0,
  'col_8': 'per MMBtu',
  'col_12': 3